In [ ]:
# Khai báo những thư viện cần thiết
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.layers import Input, Embedding, Dense, Flatten, Activation, Dropout, concatenate
from keras.layers.advanced_activations import ReLU
from keras.layers.normalization import BatchNormalization
from keras.models import Model
from keras.utils import to_categorical, plot_model
from sklearn.preprocessing import PolynomialFeatures
import matplotlib.pyplot as plt
import time

print("Bắt đầu.")
# Đường dẫn đến 2 file train và test
TRAIN_FILE_PATH = r'/content/drive/My Drive/Colab Notebooks/10set (đủ mã)/AMD_Benign_ver2(tong hop)/train-0.csv'

TEST_FILE_PATH = r'/content/drive/My Drive/Colab Notebooks/10set (đủ mã)/AMD_Benign_ver2(tong hop)/file-0.csv'
VAL_FILE_PATH = r'/content/drive/My Drive/Colab Notebooks/10set (đủ mã)/AMD_Benign_ver2(tong hop)/file-1.csv'
#FILE_TOTAL_PATH = r'F:\Thay Thuan\data\image128_per_api_drebin_benign\Drebin_Benign_128(Img_Per_API)_ver3.csv'
#tt_data = pd.read_csv(FILE_TOTAL_PATH, header=None, skiprows = 1)            #Test thử cùng với row 48 để chạy đủ file total
# Khai báo những tham số cần thiết
SIZE = 128
N_CLASSES = 228
N_EPOCHS = 50
N_BATCH_SIZE = 32
N_PERMISSION_COLUMNS = 877     #drebin 829  AMD 879    và all 908
N_API_COLUMNS = 1000
N_IMAGE_COLUMNS = 16384

# Đọc dữ liệu train và test
train_data = pd.read_csv(TRAIN_FILE_PATH, header=None, skiprows = 1)
#train_data=train_data.sample(100) 
train_data.dropna(how='any', axis=0)
test_data = pd.read_csv(TEST_FILE_PATH, header=None, skiprows = 1)
#test_data=test_data.sample(100)
test_data.dropna(how='any', axis=0)
val_data = pd.read_csv(VAL_FILE_PATH,header=None, skiprows = 1)
#val_data=val_data.sample(100)
val_data.dropna(how='any', axis=0)

# #test_data = np.concatenate((test_data,val_data)) 
# train_data = pd.concat([train_data, val_data])
# #train_data = pd.concat([train_data, test_data])
# # Tổng số cột của một dòng
# cols = train_data.columns.values
# #train_data=tt_data.sample(frac = 0.9) 
# #test_data=tt_data.sample(frac = 0.1)  #Test thử cùng với row 20,21 để chạy đủ file total
# print("ab=",test_data.shape)
# # Cột đầu tiên là label, tên cột là '2'.
# # Những cột còn lại đóng vai trò là feature
# LABEL_COLUMN = cols[2]
# FEATURE_COLUMNS = cols[3:]
# print("feature_columns = ", FEATURE_COLUMNS.shape)         #Tính tổng số đặc trưng trong file csv
# # Số lượng cột Permission feature được lấy từ cột thứ 0 đến cột N_PERMISSION_COLUMNS-1
# # Số lượng cột API feature được lấy từ cột thứ N_PERMISSION_COLUMNS đến hết.
# #TH1: sử dụng cho file có các giá trị ảnh
# IMAGE_COLUMNS = FEATURE_COLUMNS[0: N_IMAGE_COLUMNS]
# PERMISSION_COLUMNS = FEATURE_COLUMNS[N_IMAGE_COLUMNS: N_IMAGE_COLUMNS+N_PERMISSION_COLUMNS]
# API_COLUMNS = FEATURE_COLUMNS[N_IMAGE_COLUMNS+N_PERMISSION_COLUMNS:]
# print('PERMISSION_COLUMNS = ', PERMISSION_COLUMNS)
# print('API_COLUMNS = ', API_COLUMNS)
# print('IMAGE_COLUMNS = ', IMAGE_COLUMNS)
# '''
# #TH2: Sử dụng cho các file không có giá trị ảnh
# #IMAGE_COLUMNS = FEATURE_COLUMNS[0: N_IMAGE_COLUMNS]
# PERMISSION_COLUMNS = FEATURE_COLUMNS[:N_PERMISSION_COLUMNS]
# API_COLUMNS = FEATURE_COLUMNS[N_PERMISSION_COLUMNS:N_PERMISSION_COLUMNS+1833]

# print("PERMISSION_COLUMNS: ",PERMISSION_COLUMNS.shape)
# print("API_COLUMNS: ",API_COLUMNS.shape)
# # In ra thông tin các cột đã lấy
# #print('PERMISSION_COLUMNS = ', PERMISSION_COLUMNS)
# #print('API_COLUMNS = ', API_COLUMNS)
# '''



# # Hàm này để xử lý data trước khi thực hiện các bước sau
# def preprocessing():
#     all_data = pd.concat([train_data, test_data])
#     # Cột đầu tiên (cột số 2) được định nghĩa là label
#     all_data[LABEL_COLUMN] = all_data[LABEL_COLUMN].astype(int)
#     y = all_data[LABEL_COLUMN].values
#     print("y:",y)
#     all_data.pop(0) # remove file name
#     all_data.pop(1) # remove  label name
#     all_data.pop(LABEL_COLUMN) # remove label id
#     print("gia tri lon nhan cua nhan:", max(y))

#     # Chia data train và data test từ data đã đưa vào
#     train_size = len(train_data)
#     x_train = all_data.iloc[:train_size]
#     y_train = y[:train_size]
#     x_test = all_data.iloc[train_size:]
#     y_test = y[train_size:]

#     print(x_train.shape)

#     # Chia tập x_train, x_test thành 2 tập con là per và api
#     x_train_per = np.array(x_train[PERMISSION_COLUMNS])
#     x_train_api = np.array(x_train[API_COLUMNS])
#     x_test_per = np.array(x_test[PERMISSION_COLUMNS])
#     x_test_api = np.array(x_test[API_COLUMNS])
#     print("train per:", x_train_per.shape)
#     #Đầu vào dành cho wide
#     x_train_per_w = np.array(x_train[N_IMAGE_COLUMNS])
#     x_train_api_w = np.array(x_train[PERMISSION_COLUMNS+API_COLUMNS])
#     x_test_per_w = np.array(x_test[N_IMAGE_COLUMNS])
#     x_test_api_w = np.array(x_test[PERMISSION_COLUMNS+API_COLUMNS])
#     print("train per for wide:", x_train_per_w.shape)

#     return (x_train, y_train, x_test, y_test, x_train_per, x_test_per, x_train_api, x_test_api, x_train_per_w, x_test_per_w, x_train_api_w, x_test_api_w, all_data)


# def plot_graphs(history):
#     plt.plot(history.history['accuracy'])
#     plt.xlabel("Epochs")
#     plt.ylabel('accuracy')
#     plt.legend(['accuracy'])
#     plt.savefig('plot_graphs.png')
#     plt.close()


# class Wide_and_Deep:
#     # Định nghĩa class Wide_and_Deep và khai báo, khởi tạo những property cần thiết
#     def __init__(self, mode='wide and deep'):
#         self.mode = mode

#         x_train, y_train, x_test, y_test, x_train_per, x_test_per, x_train_api, x_test_api, x_train_per_w, x_test_per_w, x_train_api_w, x_test_api_w, all_data = preprocessing()
#         self.x_train = x_train
#         self.y_train = y_train
#         self.x_test = x_test
#         self.y_test = y_test
#         self.x_train_per = x_train_per
#         self.x_test_per = x_test_per
#         self.x_train_api = x_train_api
#         self.x_test_api = x_test_api
#         self.x_train_per_w = x_train_per_w
#         self.x_test_per_w = x_test_per_w
#         self.x_train_api_w = x_train_api_w
#         self.x_test_api_w = x_test_api_w
#         self.all_data = all_data
#         self.poly = PolynomialFeatures(degree=2, interaction_only=True)

#         '''self.x_train_per_poly = self.poly.fit_transform(x_train_per)
#         self.x_test_per_poly = self.poly.transform(x_test_per)
#         self.x_train_api_poly = self.poly.fit_transform(x_train_api)
#         self.x_test_api_poly = self.poly.transform(x_test_api)
# '''
#         self.per_input = None
#         self.api_input = None
#         self.deep_component_outlayer = None
#         self.logistic_input = None
#         self.logistic_per_input = None
#         self.logistic_api_input = None
#         # self.logistic_api_input = None
#         self.model = None
#         self.history = None

#     # Hàm này để tạo deep model
#     def deep_component(self):
#         per_inputs = []
#         api_inputs = []
#         per_embeds = []
#         api_embeds = []
#         dims_per = []
#         dims_api = []

#         for i in range(len(PERMISSION_COLUMNS)):
#             input_i = Input(shape=(1,), dtype='int32')
#             dim = len(np.unique(self.all_data[PERMISSION_COLUMNS[i]]))
#             embed_dim = int(np.ceil(dim ** 0.25))
#             dims_per.append(embed_dim)
#             embed_i = Embedding(dim, embed_dim, input_length=1)(input_i)
#             flatten_i = Flatten()(embed_i)
#             per_inputs.append(input_i)
#             per_embeds.append(flatten_i)

#         for i in range(len(API_COLUMNS)):
#             input_i = Input(shape=(1,), dtype='int32')
#             dim = len(np.unique(self.all_data[API_COLUMNS[i]]))
#             embed_dim = int(np.ceil(dim ** 0.25))
#             dims_api.append(embed_dim)
#             embed_i = Embedding(dim, embed_dim, input_length=1)(input_i)
#             flatten_i = Flatten()(embed_i)
#             api_inputs.append(input_i)
#             api_embeds.append(flatten_i)

#         print(per_embeds)
#         print(api_embeds)
#         print(dims_per)
#         print(dims_api)

#         # Đưa input per vào model
#         deep_per_input = Input(shape=(N_PERMISSION_COLUMNS,))
#         deep_per_dense = Dense(128, use_bias=False)(deep_per_input)

#         # Đưa input api vào model
#         deep_api_input = Input(shape=(N_API_COLUMNS,))
#         deep_api_dense = Dense(128, use_bias=False)(deep_api_input)

#         # Sau khi các input đã đi qua Full connected thì merge lại với nhau
#         # concat_embeds = concatenate(per_embeds + api_embeds)
#         concat_embeds = concatenate([deep_per_dense] + [deep_api_dense])
#         concat_embeds = Activation('relu')(concat_embeds)
#         bn_concat = BatchNormalization()(concat_embeds)

#         # Tiến hành cho qua những lớp ẩn
#         fc1 = Dense(2048, use_bias=False)(bn_concat)
#         ac1 = ReLU()(fc1)
#         bn1 = BatchNormalization()(ac1)
#         fc2 = Dense(1024, use_bias=False)(bn1)
#         ac2 = ReLU()(fc2)
#         bn2 = BatchNormalization()(ac2)
#         fc3 = Dense(512, use_bias=False)(bn2)
#         ac3 = ReLU()(fc3)
#         bn3 = BatchNormalization()(ac3)
#         fc4 = Dense(256)(bn3)
#         dropout = Dropout(0.25)
#         ac4 = ReLU()(fc4)

#         self.per_input = deep_per_input
#         self.api_input = deep_api_input

#         # Đầu ra cuối cùng của deep model
#         self.deep_component_outlayer = ac4
#         print('Finish create deep model')

#     # Hàm này để tạo wide model
#     def wide_model(self):
#         # Đưa input per vào model
#         dim_per = self.x_train_per_w.shape[1]
#         print("x_train_per_w = ", self.x_train_per_w.shape)
#         print("x_train_per = ", self.x_train_per.shape)
#         self.logistic_per_input = Input(shape=(dim_per,))
#         print("logistic_per_input = ",self.logistic_per_input.shape)
#         # Đưa input api vào model
#         dim_api = self.x_train_api_w.shape[1]
#         print("x_train_api_w = ", self.x_train_api_w.shape)
#         self.logistic_api_input = Input(shape=(dim_api,))
#         print("logistic_api_input = ",self.logistic_api_input.shape)

#     # Hàm này thực hiện merge wide & deep model lại với nhau
#     def create_model(self):
#         print('Starting create model ...')
#         self.deep_component()
#         self.wide_model()

#         if self.mode == 'wide and deep':
#             # Output layer của model cần phải có các thông tin output của từng model đã tạo
#             out_layer = concatenate([self.deep_component_outlayer, self.logistic_per_input, self.logistic_api_input])
#             # Inputs cũng cần phải được đưa vào những thông tin đúng thứ tự đã định nghĩa của từng model
#             # Nên xem summary để biết thứ tự đưa vào các input hoặc xem thứ tự gọi các lệnh input của từng model
#             inputs = [self.per_input] + [self.api_input] + [self.logistic_per_input] + [self.logistic_api_input]
#         elif self.mode == 'deep':
#             out_layer = self.deep_component_outlayer
#             inputs = [self.per_input] + [self.api_input]
#         else:
#             print('wrong mode')
#             return

#         # Output cuối cùng của model
#         output = Dense(N_CLASSES, activation='softmax')(out_layer)
#         print(f'input = {len(inputs)}')

#         # Tạo model với thông tin inputs và output đã định nghĩa
#         self.model = Model(inputs=inputs, outputs=output)
#         print(self.model.summary())

#     # Hàm này dùng để train model, ứng thứ tự các input đã khai báo, cần đưa vào model đúng thứ tự tập input train
#     def train_model(self):
#         if not self.model:
#             print('You have to create model first')
#             return

#         if self.mode == 'wide and deep':
#             print("chạy vào train wide and deep")
#             input_data = [self.x_train_per] + [self.x_train_api] + [self.x_train_per_w] + [self.x_train_api_w]
#             print("xxxxxxxx")
#         elif self.mode == 'deep':
#             input_data = [self.x_train_per] + [self.x_train_api]
#         else:
#             print('wrong mode')
#             return

#         # Do bài toán ở đây là multi classification nên loss phải là 'categorical_crossentropy'
#         self.model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
#         print("qua compile")
#         # Chuyển đối số chiều của tập y_train từ [len(y_train), None] về [len(y_train), N_CLASSES]
#         self.y_train = to_categorical(self.y_train, N_CLASSES)
#         # Định nghĩa callbacks theo 'accuracy'
#         es = tf.keras.callbacks.EarlyStopping(monitor='accuracy', mode='max')
#         # callbacks = [es]
#         callbacks = []
#         # Trong quá trình train có thể sử dụng callbacks hoặc không.
#         self.history = self.model.fit(input_data, self.y_train, epochs=N_EPOCHS, batch_size=N_BATCH_SIZE)
#     # Hàm này dùng để đánh giá kết quả model đã train bằng tập test
#     # Các tham số đưa vào được tính toán tương tự với khi train
#     def evaluate_model(self):
#         if not self.model:
#             print('You have to create model first')
#             return

#         if self.mode == 'wide and deep':
#             print("chạy vào test wide and deep")
#             input_data = [self.x_test_per] + [self.x_test_api] + [self.x_test_per_w] + [self.x_test_api_w]
#         elif self.mode == 'deep':
#             input_data = [self.x_test_per] + [self.x_test_api]

#         else:
#             print('wrong mode')
#             return

#         self.y_test = to_categorical(self.y_test,N_CLASSES)
#         loss, acc = self.model.evaluate(input_data, self.y_test)
#         print(f'\ntest_loss: {loss} - test_acc: {acc}')

#     # Lưu lại model dưới dạng .h5 (keras output format) để sử dụng về sau
#     def save_model(self, filename='wide_and_deep.h5'):
#         self.model.save(filename)


# if __name__ == '__main__':
#     # Khai báo 1 đối tượng Wide_and_Deep() và sử dụng các hàm đã định nghĩa của Wide_and_Deep
#     t0 = time.time()
#     print("bắt đầu chạy winde and deep")
#     wide_deep_net = Wide_and_Deep()
#     t1 = time.time()
#     print("chạy xong wide and deep")
#     print("bắt đầu chạy create model")
#     wide_deep_net.create_model()
#     print("chạy xong create model")
#     print("bắt đầu chạy train model")
#     wide_deep_net.train_model()
#     print("chạy xong train model")
#     print("bắt đầu chạy evaluate_model")
#     wide_deep_net.evaluate_model()
#     print("chạy xong evaluate_model")
#     wide_deep_net.save_model()

#     #plot_graphs(wide_deep_net.history)

#     # Lưu lại thông tin của model dứa dạng file .png
#     #plot_model(wide_deep_net.model, to_file='model.png', show_shapes=True, show_layer_names=False)
#     t2 = time.time()

#     print("thoi gian chạy model không tính comatrix: %.2f phút." %((t2-t1)/60))
#     print("thoi gian chạy xong model: %.2f phút." %((t2-t0)/60))
#     print('Hoàn thành việc huấn luyện và kiểm tra!')
    

Bắt đầu.


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,18224,18225,18226,18227,18228,18229,18230,18231,18232,18233,18234,18235,18236,18237,18238,18239,18240,18241,18242,18243,18244,18245,18246,18247,18248,18249,18250,18251,18252,18253,18254,18255,18256,18257,18258,18259,18260,18261,18262,18263
0,Diner Dash Apk Mod Unlock All,0_apps,0,68.279,12.9356,7.9992,161.953,90.7047,90.5023,43.3189,98.713,2.280,2.348,73.0943,54.1655,69.7622,70.0623,70.7757,79.3607,93.0613,60.3778,148.9597,135.5679,113.0776,172.6493,190.3746,49.0576,127.5624,164.7496,126.1370,167.7732,20.9380,134.4906,182.5286,145.8025,91.4967,173.5355,126.8305,143.6752,162.8809,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,DINO HUNTER DEADLY SHORES Unlock All,0_apps,0,68.279,12.9356,7.9992,36.389,60.6697,0.0000,0.0000,0.000,2.166,0.000,95.3985,111.2016,86.1470,106.1420,110.9179,96.6154,124.6242,78.2525,60.1245,61.1080,121.6286,104.5150,173.5629,123.6264,16.9599,60.9775,108.5904,74.8190,161.7828,91.8240,50.6479,153.4492,181.9995,162.5386,82.2087,101.6046,201.0272,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Dino Island Apk Mod Unlock All,0_apps,0,68.279,12.9356,7.9992,134.056,91.2973,110.0761,34.9497,12.897,2.280,2.348,73.0943,54.1655,69.7622,70.0623,70.7757,79.3607,93.0613,60.3778,195.7305,144.8338,35.0360,164.1571,138.2036,31.6100,103.9312,55.4769,81.3985,143.4723,167.2080,64.4058,40.2014,138.1254,144.7481,98.4316,170.0235,241.1530,100.8253,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Dino Pets Apk Mod All Unlocked,0_apps,0,68.279,12.9356,7.9992,51.282,119.1237,113.5027,51.3892,80.157,2.280,2.348,73.0943,54.1655,69.7622,70.0623,70.7757,79.3607,93.0613,60.3778,162.5427,144.8338,112.2070,137.8222,153.2237,119.3591,130.8517,30.6592,70.4840,142.7514,110.6124,57.8580,68.2250,70.4434,103.4844,65.1752,118.3256,139.7709,209.1891,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Dino Safari 2 Pro Apk Mod Unlock All,0_apps,0,68.279,12.9356,7.9992,67.104,70.4868,111.2174,57.6661,59.253,2.280,2.348,73.0943,54.1655,69.7622,70.0623,70.7757,79.3607,93.0613,60.3778,147.7227,145.1327,112.2070,161.2542,80.6569,50.2735,130.8517,78.4832,167.5080,158.1221,170.1749,118.7560,180.6364,224.3023,95.1140,105.7334,58.0356,167.4293,191.5817,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2580,add31a0015c25a4a1a8c543888673765,Youmi,225,68.279,12.9356,3.3032,10.595,53.3881,0.0000,0.0000,0.000,2.280,0.000,73.0943,54.1655,69.7622,70.0623,70.7757,79.3607,84.7393,106.0646,116.0987,158.3191,174.2275,96.3861,77.9510,83.8452,120.6949,26.5397,82.3044,175.9906,141.2353,217.7750,165.5598,47.2137,198.9119,123.7500,168.0489,110.8445,60.6109,...,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,1,1,0,1,0,0,0,0,0,0,0
2581,adba41f4996dbbbbba69ad91a7b99fd4,Youmi,225,68.279,12.9356,3.3032,56.150,76.3517,72.1758,74.1272,28.404,2.850,2.348,106.4716,85.7443,109.9043,104.2022,91.8253,111.1952,112.3694,96.6154,204.4072,10.6020,162.4206,41.1746,176.8798,87.7683,76.9865,35.3597,125.3470,171.8497,160.8812,163.1262,108.5710,115.0606,191.3958,137.4708,41.9799,151.2514,114.1468,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0
2582,b481c8c6bbcf1ac1d39be46cfd602977,Ztorg,226,68.279,12.9356,7.9992,21.629,75.9136,0.0000,0.0000,0.000,2.280,2.348,73.0943,54.1655,69.7622,70.0623,70.7757,79.3607,93.0613,60.3778,128.6709,147.5284,133.5372,182.7511,74.1451,66.3797,32.1327,69.4231,84.5468,96.5868,154.7157,45.7992,140.8460,114.

In [ ]:
#test_data = np.concatenate((test_data,val_data)) 
train_data = pd.concat([train_data, val_data])
#train_data = pd.concat([train_data, test_data])
# Tổng số cột của một dòng
cols = train_data.columns.values
#train_data=tt_data.sample(frac = 0.9) 
#test_data=tt_data.sample(frac = 0.1)  #Test thử cùng với row 20,21 để chạy đủ file total
print("ab=",test_data.shape)
# Cột đầu tiên là label, tên cột là '2'.
# Những cột còn lại đóng vai trò là feature
LABEL_COLUMN = cols[2]
FEATURE_COLUMNS = cols[3:]
print("feature_columns = ", FEATURE_COLUMNS.shape)         #Tính tổng số đặc trưng trong file csv
# Số lượng cột Permission feature được lấy từ cột thứ 0 đến cột N_PERMISSION_COLUMNS-1
# Số lượng cột API feature được lấy từ cột thứ N_PERMISSION_COLUMNS đến hết.
#TH1: sử dụng cho file có các giá trị ảnh
IMAGE_COLUMNS = FEATURE_COLUMNS[0: N_IMAGE_COLUMNS]
PERMISSION_COLUMNS = FEATURE_COLUMNS[N_IMAGE_COLUMNS: ]
API_COLUMNS = FEATURE_COLUMNS[N_IMAGE_COLUMNS+N_PERMISSION_COLUMNS:]
print('PERMISSION_COLUMNS = ', PERMISSION_COLUMNS)
print('API_COLUMNS = ', API_COLUMNS)
print('IMAGE_COLUMNS = ', IMAGE_COLUMNS)
'''
#TH2: Sử dụng cho các file không có giá trị ảnh
#IMAGE_COLUMNS = FEATURE_COLUMNS[0: N_IMAGE_COLUMNS]
PERMISSION_COLUMNS = FEATURE_COLUMNS[:N_PERMISSION_COLUMNS]
API_COLUMNS = FEATURE_COLUMNS[N_PERMISSION_COLUMNS:N_PERMISSION_COLUMNS+1833]

print("PERMISSION_COLUMNS: ",PERMISSION_COLUMNS.shape)
print("API_COLUMNS: ",API_COLUMNS.shape)
# In ra thông tin các cột đã lấy
#print('PERMISSION_COLUMNS = ', PERMISSION_COLUMNS)
#print('API_COLUMNS = ', API_COLUMNS)
'''



# Hàm này để xử lý data trước khi thực hiện các bước sau
def preprocessing():
    all_data = pd.concat([train_data, test_data])
    # Cột đầu tiên (cột số 2) được định nghĩa là label
    all_data[LABEL_COLUMN] = all_data[LABEL_COLUMN].astype(int)
    y = all_data[LABEL_COLUMN].values
    print("y:",y)
    all_data.pop(0) # remove file name
    all_data.pop(1) # remove  label name
    all_data.pop(LABEL_COLUMN) # remove label id
    print("gia tri lon nhan cua nhan:", max(y))

    # Chia data train và data test từ data đã đưa vào
    train_size = len(train_data)
    x_train = all_data.iloc[:train_size]
    y_train = y[:train_size]
    x_test = all_data.iloc[train_size:]
    y_test = y[train_size:]

    print(x_train.shape)

    # Chia tập x_train, x_test thành 2 tập con là per và api
    x_train_per = np.array(x_train[PERMISSION_COLUMNS])
    x_train_api = np.array(x_train[API_COLUMNS])
    x_test_per = np.array(x_test[PERMISSION_COLUMNS])
    x_test_api = np.array(x_test[API_COLUMNS])
    print("train per:", x_train_per.shape)
    #Đầu vào dành cho wide
    x_train_per_w = np.array(x_train[N_IMAGE_COLUMNS])
    x_train_api_w = np.array(x_train[PERMISSION_COLUMNS.tolist()+API_COLUMNS.tolist()])
    x_test_per_w = np.array(x_test[N_IMAGE_COLUMNS])
    x_test_api_w = np.array(x_test[PERMISSION_COLUMNS.tolist()+API_COLUMNS.tolist()])
    print("train per for wide:", x_train_per_w.shape)

    return (x_train, y_train, x_test, y_test, x_train_per, x_test_per, x_train_api, x_test_api, x_train_per_w, x_test_per_w, x_train_api_w, x_test_api_w, all_data)


def plot_graphs(history):
    plt.plot(history.history['accuracy'])
    plt.xlabel("Epochs")
    plt.ylabel('accuracy')
    plt.legend(['accuracy'])
    plt.savefig('plot_graphs.png')
    plt.close()


class Wide_and_Deep:
    # Định nghĩa class Wide_and_Deep và khai báo, khởi tạo những property cần thiết
    def __init__(self, mode='wide and deep'):
        self.mode = mode

        x_train, y_train, x_test, y_test, x_train_per, x_test_per, x_train_api, x_test_api, x_train_per_w, x_test_per_w, x_train_api_w, x_test_api_w, all_data = preprocessing()
        self.x_train = x_train
        self.y_train = y_train
        self.x_test = x_test
        self.y_test = y_test
        self.x_train_per = x_train_per
        self.x_test_per = x_test_per
        self.x_train_api = x_train_api
        self.x_test_api = x_test_api
        self.x_train_per_w = x_train_per_w
        self.x_test_per_w = x_test_per_w
        self.x_train_api_w = x_train_api_w
        self.x_test_api_w = x_test_api_w
        self.all_data = all_data
        self.poly = PolynomialFeatures(degree=2, interaction_only=True)

        '''self.x_train_per_poly = self.poly.fit_transform(x_train_per)
        self.x_test_per_poly = self.poly.transform(x_test_per)
        self.x_train_api_poly = self.poly.fit_transform(x_train_api)
        self.x_test_api_poly = self.poly.transform(x_test_api)
'''
        self.per_input = None
        self.api_input = None
        self.deep_component_outlayer = None
        self.logistic_input = None
        self.logistic_per_input = None
        self.logistic_api_input = None
        # self.logistic_api_input = None
        self.model = None
        self.history = None

    # Hàm này để tạo deep model
    def deep_component(self):
        per_inputs = []
        api_inputs = []
        per_embeds = []
        api_embeds = []
        dims_per = []
        dims_api = []

        for i in range(len(PERMISSION_COLUMNS)):
            input_i = Input(shape=(1,), dtype='int32')
            dim = len(np.unique(self.all_data[PERMISSION_COLUMNS[i]]))
            embed_dim = int(np.ceil(dim ** 0.25))
            dims_per.append(embed_dim)
            embed_i = Embedding(dim, embed_dim, input_length=1)(input_i)
            flatten_i = Flatten()(embed_i)
            per_inputs.append(input_i)
            per_embeds.append(flatten_i)

        for i in range(len(API_COLUMNS)):
            input_i = Input(shape=(1,), dtype='int32')
            dim = len(np.unique(self.all_data[API_COLUMNS[i]]))
            embed_dim = int(np.ceil(dim ** 0.25))
            dims_api.append(embed_dim)
            embed_i = Embedding(dim, embed_dim, input_length=1)(input_i)
            flatten_i = Flatten()(embed_i)
            api_inputs.append(input_i)
            api_embeds.append(flatten_i)

        print(per_embeds)
        print(api_embeds)
        print(dims_per)
        print(dims_api)

        # Đưa input per vào model
        deep_per_input = Input(shape=(N_PERMISSION_COLUMNS,))
        deep_per_dense = Dense(128, use_bias=False)(deep_per_input)

        # Đưa input api vào model
        deep_api_input = Input(shape=(N_API_COLUMNS,))
        deep_api_dense = Dense(128, use_bias=False)(deep_api_input)

        # Sau khi các input đã đi qua Full connected thì merge lại với nhau
        # concat_embeds = concatenate(per_embeds + api_embeds)
        concat_embeds = concatenate([deep_per_dense] + [deep_api_dense])
        concat_embeds = Activation('relu')(concat_embeds)
        bn_concat = BatchNormalization()(concat_embeds)

        # Tiến hành cho qua những lớp ẩn
        fc1 = Dense(2048, use_bias=False)(bn_concat)
        ac1 = ReLU()(fc1)
        bn1 = BatchNormalization()(ac1)
        fc2 = Dense(1024, use_bias=False)(bn1)
        ac2 = ReLU()(fc2)
        bn2 = BatchNormalization()(ac2)
        fc3 = Dense(512, use_bias=False)(bn2)
        ac3 = ReLU()(fc3)
        bn3 = BatchNormalization()(ac3)
        fc4 = Dense(256)(bn3)
        dropout = Dropout(0.25)
        ac4 = ReLU()(fc4)

        self.per_input = deep_per_input
        self.api_input = deep_api_input

        # Đầu ra cuối cùng của deep model
        self.deep_component_outlayer = ac4
        print('Finish create deep model')

    # Hàm này để tạo wide model
    def wide_model(self):
        # Đưa input per vào model
        dim_per = self.x_train_per_w.shape
        print("x_train_per_w = ", self.x_train_per_w.shape)
        print("x_train_per = ", self.x_train_per.shape)
        self.logistic_per_input = Input(shape=(dim_per,))
        print("logistic_per_input = ",self.logistic_per_input.shape)
        # Đưa input api vào model
        dim_api = self.x_train_api_w.shape[1]
        print("x_train_api_w = ", self.x_train_api_w.shape)
        self.logistic_api_input = Input(shape=(dim_api,))
        print("logistic_api_input = ",self.logistic_api_input.shape)

    # Hàm này thực hiện merge wide & deep model lại với nhau
    def create_model(self):
        print('Starting create model ...')
        self.deep_component()
        self.wide_model()

        if self.mode == 'wide and deep':
            # Output layer của model cần phải có các thông tin output của từng model đã tạo
            out_layer = concatenate([self.deep_component_outlayer, self.logistic_per_input, self.logistic_api_input])
            # Inputs cũng cần phải được đưa vào những thông tin đúng thứ tự đã định nghĩa của từng model
            # Nên xem summary để biết thứ tự đưa vào các input hoặc xem thứ tự gọi các lệnh input của từng model
            inputs = [self.per_input] + [self.api_input] + [self.logistic_per_input] + [self.logistic_api_input]
        elif self.mode == 'deep':
            out_layer = self.deep_component_outlayer
            inputs = [self.per_input] + [self.api_input]
        else:
            print('wrong mode')
            return

        # Output cuối cùng của model
        output = Dense(N_CLASSES, activation='softmax')(out_layer)
        print(f'input = {len(inputs)}')

        # Tạo model với thông tin inputs và output đã định nghĩa
        self.model = Model(inputs=inputs, outputs=output)
        print(self.model.summary())

    # Hàm này dùng để train model, ứng thứ tự các input đã khai báo, cần đưa vào model đúng thứ tự tập input train
    def train_model(self):
        if not self.model:
            print('You have to create model first')
            return

        if self.mode == 'wide and deep':
            print("chạy vào train wide and deep")
            input_data = [self.x_train_per] + [self.x_train_api] + [self.x_train_per_w] + [self.x_train_api_w]
            print("xxxxxxxx")
        elif self.mode == 'deep':
            input_data = [self.x_train_per] + [self.x_train_api]
        else:
            print('wrong mode')
            return

        # Do bài toán ở đây là multi classification nên loss phải là 'categorical_crossentropy'
        self.model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
        print("qua compile")
        # Chuyển đối số chiều của tập y_train từ [len(y_train), None] về [len(y_train), N_CLASSES]
        self.y_train = to_categorical(self.y_train, N_CLASSES)
        # Định nghĩa callbacks theo 'accuracy'
        es = tf.keras.callbacks.EarlyStopping(monitor='accuracy', mode='max')
        # callbacks = [es]
        callbacks = []
        # Trong quá trình train có thể sử dụng callbacks hoặc không.
        self.history = self.model.fit(input_data, self.y_train, epochs=N_EPOCHS, batch_size=N_BATCH_SIZE)
    # Hàm này dùng để đánh giá kết quả model đã train bằng tập test
    # Các tham số đưa vào được tính toán tương tự với khi train
    def evaluate_model(self):
        if not self.model:
            print('You have to create model first')
            return

        if self.mode == 'wide and deep':
            print("chạy vào test wide and deep")
            input_data = [self.x_test_per] + [self.x_test_api] + [self.x_test_per_w] + [self.x_test_api_w]
        elif self.mode == 'deep':
            input_data = [self.x_test_per] + [self.x_test_api]

        else:
            print('wrong mode')
            return

        self.y_test = to_categorical(self.y_test,N_CLASSES)
        loss, acc = self.model.evaluate(input_data, self.y_test)
        print(f'\ntest_loss: {loss} - test_acc: {acc}')

    # Lưu lại model dưới dạng .h5 (keras output format) để sử dụng về sau
    def save_model(self, filename='wide_and_deep.h5'):
        self.model.save(filename)


if __name__ == '__main__':
    # Khai báo 1 đối tượng Wide_and_Deep() và sử dụng các hàm đã định nghĩa của Wide_and_Deep
    t0 = time.time()
    print("bắt đầu chạy winde and deep")
    wide_deep_net = Wide_and_Deep()
    t1 = time.time()
    print("chạy xong wide and deep")
    print("bắt đầu chạy create model")
    wide_deep_net.create_model()
    print("chạy xong create model")
    print("bắt đầu chạy train model")
    wide_deep_net.train_model()
    print("chạy xong train model")
    print("bắt đầu chạy evaluate_model")
    wide_deep_net.evaluate_model()
    print("chạy xong evaluate_model")
    wide_deep_net.save_model()

    #plot_graphs(wide_deep_net.history)

    # Lưu lại thông tin của model dứa dạng file .png
    #plot_model(wide_deep_net.model, to_file='model.png', show_shapes=True, show_layer_names=False)
    t2 = time.time()

    print("thoi gian chạy model không tính comatrix: %.2f phút." %((t2-t1)/60))
    print("thoi gian chạy xong model: %.2f phút." %((t2-t0)/60))
    print('Hoàn thành việc huấn luyện và kiểm tra!')

ab= (2585, 18264)
feature_columns =  (18261,)
PERMISSION_COLUMNS =  [16387 16388 16389 ... 18261 18262 18263]
API_COLUMNS =  [17264 17265 17266 17267 17268 17269 17270 17271 17272 17273 17274 17275
 17276 17277 17278 17279 17280 17281 17282 17283 17284 17285 17286 17287
 17288 17289 17290 17291 17292 17293 17294 17295 17296 17297 17298 17299
 17300 17301 17302 17303 17304 17305 17306 17307 17308 17309 17310 17311
 17312 17313 17314 17315 17316 17317 17318 17319 17320 17321 17322 17323
 17324 17325 17326 17327 17328 17329 17330 17331 17332 17333 17334 17335
 17336 17337 17338 17339 17340 17341 17342 17343 17344 17345 17346 17347
 17348 17349 17350 17351 17352 17353 17354 17355 17356 17357 17358 17359
 17360 17361 17362 17363 17364 17365 17366 17367 17368 17369 17370 17371
 17372 17373 17374 17375 17376 17377 17378 17379 17380 17381 17382 17383
 17384 17385 17386 17387 17388 17389 17390 17391 17392 17393 17394 17395
 17396 17397 17398 17399 17400 17401 17402 17403 17404 17405 17406 17407

ValueError: ignored